# 🏥 Care AI: An Empathetic Virtual Nurse Simulator - Fine-Tuned LLM

## 📌 Project Overview
**Care AI** is a specialized behavioral simulator designed to act as a **Virtual AI Nurse**. Unlike generic chatbots, this model has been fine-tuned to provide empathetic, safe, and accurate medical triage advice to patients.

### 🎯 Key Features
* **Roleplay Engine:** Acts as a professional nurse, adapting its tone based on the patient (e.g., gentle for children, authoritative for medication errors).
* **Efficient Fine-Tuning:** Uses **LoRA (Low-Rank Adaptation)** to specialize a lightweight `flan-t5-small` model without high compute costs.
* **Dynamic Response Generation:** Implements advanced sampling parameters (temperature, top-p) to generate natural, human-like responses instead of robotic outputs.

### 🛠️ Tech Stack
* **Model:** `google/flan-t5-small`
* **Library:** Hugging Face `transformers` & `peft`
* **Training:** Supervised Fine-Tuning (SFT) on `nurse_patient_conversations.jsonl`

### ⚙️ 1. Imports & Setup
Here, we import the necessary libraries for building the pipeline. 
* **`transformers`**: For loading the pre-trained T5 model and tokenizer.
* **`peft`**: For implementing LoRA (Low-Rank Adaptation) to fine-tune efficiently.
* **`datasets`**: To handle the JSONL conversation data.
* Other Installed and imported the following libraries for initial serup are ->
    - numpy
    - pandas
    - matplotlib
    - os
    - pdfplumber
    - nltk
    - pathlib
    - json
    - openai
    - tqdm
    - transformers
    - torch
    - peft
    - re
    - datasets
    - faiss
    - pickle
    - sentence_transformers

In [1]:
# ==========================
# 1) Imports & Setup
# ==========================
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import pdfplumber
import nltk
from pathlib import Path
import json
from openai import OpenAI
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, TrainingArguments, get_scheduler, BitsAndBytesConfig, LlamaTokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, AutoModel, Trainer     
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from tqdm import tqdm
import re
from datasets import load_dataset
import faiss
import pickle
from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/spy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 📂 2. Data Preparation & Preprocessing
We load the `nurse_patient_conversations.jsonl` dataset. 

The model needs to learn a specific input-output format:
* **Input:** The System Instruction + The Patient's Query.
* **Target:** The Nurse's Empathetic Response.

We preprocess the data to combine the `instruction` and `input` fields into a single string for the model to read.

In [7]:
# Load your JSONL file
# Assumes each line is like: {"instruction": "...", "input": "...", "output": "..."}
dataset_path = "nurse_patient_conversations.jsonl"  # <-- replace with your file path
dataset = load_dataset("json", data_files=dataset_path, split="train")

# Combine instruction + input as model input, output as target
def preprocess(example):
    input_text = example["instruction"]
    if example.get("input"):
        input_text += "\n" + example["input"]
    example["input_text"] = input_text
    example["target_text"] = example["output"]
    return example

dataset = dataset.map(preprocess)

Generating train split: 50000 examples [00:00, 1051546.62 examples/s]
Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 59003.95 examples/s]


### 🧠 3. Model Initialization & Tokenization
We use **`google/flan-t5-small`** as our base. It is a "Sequence-to-Sequence" model, making it perfect for Q&A and Chat tasks.
* **Max Input Length:** 512 tokens (to handle detailed patient histories).
* **Max Output Length:** 128 tokens (to keep nurse responses concise).

In [9]:
# Load tokenizer and model (light model)
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [10]:
# Tokenize
max_input_length = 512
max_output_length = 128

In [11]:
def tokenize_fn(example):
    model_inputs = tokenizer(
        example["input_text"], 
        max_length=max_input_length, 
        padding="max_length", 
        truncation=True
    )
    labels = tokenizer(
        example["target_text"], 
        max_length=max_output_length, 
        padding="max_length", 
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_fn, remove_columns=dataset.column_names)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:08<00:00, 5772.68 examples/s]


### ⚡ 4. LoRA Configuration (PEFT)
Instead of retraining the entire model (which is slow and expensive), we use **LoRA (Low-Rank Adaptation)**. 
* We attach small, trainable matrices to the attention layers (`q` and `v`).
* We freeze the rest of the model.
* **Result:** We only train ~0.5% of the parameters, allowing this to run on standard hardware.

In [12]:
# 4️⃣ LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],  # for T5 model
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

### 🏋️‍♂️ 5. Training the "Nurse Persona"
We train the model for **3 Epochs** to internalize the nursing persona.
* **Batch Size:** 4 (Optimized for CPU/lower memory).
* **Learning Rate:** `3e-4` (Standard for LoRA).
* **Goal:** Minimize the loss (difference) between the model's generated response and the expert nurse responses in the dataset.

In [13]:
model = get_peft_model(model, lora_config)

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/opt/anaconda3/envs/spy/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [23]:
# Training setup
training_args = TrainingArguments(
    output_dir="./care_ai_model",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True, # Will run on CPU
    use_cpu=True,
    save_total_limit=2
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

In [25]:
# Fine-tune
trainer.train()

Step,Training Loss
10,33.286400
20,29.923100
30,24.483000
40,18.573700
50,12.438800
60,8.116500
70,6.361500
80,5.397200
90,4.914000
100,4.692200


TrainOutput(global_step=9375, training_loss=1.6461666379292805, metrics={'train_runtime': 430706.784, 'train_samples_per_second': 0.348, 'train_steps_per_second': 0.022, 'total_flos': 2.80421203968e+16, 'train_loss': 1.6461666379292805, 'epoch': 3.0})

---

## 📉 Training Performance & Metrics

The model's transition from a base LLM to an empathetic nursing persona was achieved through a 119-hour **[Around 4 days and 23 hours]** compute cycle.

### 📊 Training Results
| Metric | Value |
| :--- | :--- |
| **Total Runtime** | 4d 23h 38m (430,706 sec) |
| **Total Training Steps** | 9,375 |
| **Hardware Throughput** | 0.348 samples/sec |
| **Final Global Loss** | 1.646 |
| **Total FLOPs** | 2.80e+16 |

---

### ⏱️ Training Progression
```mermaid
gantt
    title Care AI Training Timeline (3 Epochs)
    dateFormat  X
    axisFormat %s
    section Training Phase
    Epoch 1 (Learning Dialect)    :0, 143568
    Epoch 2 (Refining Empathy)    :143568, 287136
    Epoch 3 (Safety & Nuance)     :287136, 430706
```
---

In [28]:
# Save LoRA-finetuned model
model.save_pretrained("./care_ai_model")
tokenizer.save_pretrained("./care_ai_model")

('./care_ai_model/tokenizer_config.json',
 './care_ai_model/special_tokens_map.json',
 './care_ai_model/spiece.model',
 './care_ai_model/added_tokens.json',
 './care_ai_model/tokenizer.json')

In [31]:
# Load your fine-tuned model + tokenizer
model_path = "./care_ai_model"   # change if you saved somewhere else
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

### 🗣️ 6. The Care AI Inference Engine
This custom function generates responses. Unlike standard generation, we use **Sampling** to make the AI sound human.

* **`do_sample=True`**: Enables creativity (vs. robotic deterministic answers).
* **`temperature=0.7`**: Balances randomness to ensure the nurse sounds professional but not repetitive.
* **`repetition_penalty=1.5`**: Prevents the model from getting stuck in loops or repeating phrases.

In [85]:
def care_ai_model(model, prompt, tokenizer, max_length=256):
    
    # 1. Ensure inputs are on the same device as the model (CPU/GPU)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        
        # 🟢 CRITICAL FIXES FOR VARIETY:
        do_sample=True,         # Enables "Creativity" (picks from a pool of words)
        temperature=0.7,        # Randomness control (0.7 is balanced, 1.0 is wild, 0.1 is robotic)
        top_p=0.9,              # Nucleus sampling (focuses on the top 90% likely words)
        repetition_penalty=1.5, # Penalizes the model if it repeats the same sentence
        
        # 🔴 Remove 'num_beams' or set to 1 when using sampling for chat
        num_beams=1,
        early_stopping=True
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("AI Nurse:", response)

### 🏥 7. Simulation: Testing Different Patient Scenarios
We now test the **Fine-Tuned Care AI** against various patient personas to ensure it adapts correctly.

**Scenarios:**
1.  **The Anxious Patient:** Testing for empathy and reassurance.
2.  **The Medication Error:** Testing for safety protocols and authoritative guidance.

In [61]:
# 1. Define the Persona (System Instruction)
instruction = "You are a professional, empathetic AI Nurse assisting a patient. Provide clear medical guidance and comfort."

# 2. Define the Scenario (Patient asks a question)
patient_query = "I have been feeling really dizzy and nauseous since this morning. Should I be worried?"

# 3. Construct the Prompt
prompt = f"""{instruction} Patient: {patient_query} Nurse:"""

# 4. Tokenize and generate respons and Call the model and print the decoded response 
care_ai_model(model, prompt, tokenizer)

AI Nurse: It’s okay to feel anxious. Would you like me to explain the treatment steps to make you feel more comfortable?


In [69]:
# 1. Define the Persona (System Instruction)
instruction = "You are a practical and informative AI Nurse assisting a patient with medication management. Focus on safety protocols."

# 2. Define the Scenario (Patient asks a question)
patient_query = "I forgot to take my blood pressure medication this morning and I'm feeling a little dizzy now. Should I take two pills now to make up for it?"

# 3. Construct the Prompt
prompt = f"""{instruction}
Patient: {patient_query}
Nurse:"""

# 4. Tokenize and generate respons and Call the model and print the decoded response 
care_ai_model(model, prompt, tokenizer)

AI Nurse: You should take the pill once in the morning with water, and once at night before bed. Always follow your doctor’s prescription.


### 8. Conclusion & Future Scope

### 🏆 Project Summary
In this project, I tried to built **Care AI**, a domain-specific conversational agent capable of acting as a virtual nurse. By leveraging **Parameter-Efficient Fine-Tuning (PEFT/LoRA)**, we transformed a general-purpose language model (`flan-t5-small`) into a specialized medical assistant without the need for massive computational resources.

**Key Achievements:**
* **Behavioral Adaptation:** The model successfully learned to adopt an empathetic, professional "Nurse" persona, distinct from its default generic assistant behavior.
* **Safety & Triage:** The simulation demonstrated the model's ability to handle high-risk queries (e.g., medication errors) with appropriate caution and safety warnings.
* **Efficient Architecture:** Achieved high-quality dialogue generation using <1% trainable parameters via LoRA.

### 🚀 Future Improvements
To further enhance Care AI for real-world deployment, the following steps are recommended:
1.  **Guardrails:** Implement strict output filters (e.g., NeMo Guardrails) to prevent the model from answering non-medical queries or providing dangerous advice.
2.  **Multimodal Support:** Enable the model to process images (e.g., patient uploading a photo of a rash) for more accurate preliminary triage.